# Mission to Mars

In [35]:
#Import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [36]:
#Initialize Chrome browser
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

### Scraper 1 - NASA Mars News

In [37]:
#Go to NASA site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [38]:
#Parse NASA site with BS4
html = browser.html
news_soup = BeautifulSoup(html, "lxml")
slide_element = news_soup.select_one("ul.item_list li.slide")

In [39]:
#Find news article
slide_element.find("div", class_="content_title")

<div class="content_title"><a href="/news/8551/mars-scientists-investigate-ancient-life-in-australia/" target="_self">Mars Scientists Investigate Ancient Life in Australia</a></div>

In [40]:
#Scrape news title
news_title = slide_element.find("div", class_="content_title").get_text()
print(news_title)

Mars Scientists Investigate Ancient Life in Australia


In [41]:
#Scrape the paragraph text
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. 


### Scraper 2 - Featured Image

In [42]:
#Go to JPL site
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [43]:
#Use Splinter to click on featured image
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [44]:
#Click on "more info"
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [45]:
#Parse HTML
html = browser.html
image_soup = BeautifulSoup(html, "lxml")

In [46]:
img_url = image_soup.select_one("figure.lede a img").get("src")
img_url

'/spaceimages/images/largesize/PIA17449_hires.jpg'

In [47]:
#Create image link
img_url = f"https://www.jpl.nasa.gov{img_url}"
print(img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17449_hires.jpg


### Scraper 3 - Mars Weather

In [48]:
#Go to Mars Twitter page
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [49]:
#Parse HTML
html = browser.html
weather_soup = BeautifulSoup(html, "lxml")

In [50]:
#Find weather tweet
mars_weather_tweet = weather_soup.find("div", 
                                       attrs={
                                           "class": "tweet", 
                                            "data-name": "Mars Weather"
                                        })

In [51]:
#Find weather tweet text
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
print(mars_weather)

InSight sol 348 (2019-11-19) low -102.5ºC (-152.5ºF) high -23.2ºC (-9.8ºF)
winds from the SSE at 5.1 m/s (11.5 mph) gusting to 18.9 m/s (42.3 mph)
pressure at 6.80 hPapic.twitter.com/iZwojPj9au


### Scraper 4 - Mars Facts

In [52]:
#Go to Mars fact site
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_df)
mars_df.columns=["Description", "Value"]
mars_df.set_index("Description", inplace=True)
mars_df

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [53]:
#Go to USGS Mars hemisphere page
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [54]:
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate Backwards
    browser.back()

In [55]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]